In [1]:
!pip3 show gym

<h2>Setting Path of GYM</h2>

In [2]:
import sys
sys.path.append('/home/dnclab/env/mi333/lib/python3.8/site-packages')

In [3]:
import gym
import numpy as np
import random
import math
from time import sleep

from mujoco_py import load_model_from_path, MjSim, MjViewer

/home/dnclab/anaconda3/envs/mi333/lib/python3.8/site-packages/gym/envs/registration.py:423: UserWarning: WARN: Custom namespace `ALE` is being overridden by namespace `ALE`. If you are developing a plugin you shouldn't specify a namespace in `register` calls. The namespace is specified through the entry point package metadata.
  logger.warn(


<h2>Import Cart Pole Environment</h2>

In [4]:
env = gym.make('CartPole-v0')

/home/dnclab/anaconda3/envs/mi333/lib/python3.8/site-packages/gym/envs/registration.py:568: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


<h2>Set Parameters</h2>

In [5]:
NUM_BUCKETS = (1, 1, 6, 3) # (loc, velocity, angle, angle_velocity)
NUM_ACTIONS = env.action_space.n # (left, right)
STATE_BOUNDS = list(zip(env.observation_space.low, env.observation_space.high))
STATE_BOUNDS[1] = [-0.5, 0.5]
STATE_BOUNDS[3] = [-math.radians(50), math.radians(50)]
ACTION_INDEX = len(NUM_BUCKETS)

<h2>Make a Q Table</h2>

In [6]:
q_table = np.zeros(NUM_BUCKETS + (NUM_ACTIONS,))
q_table

array([[[[[0., 0.],
          [0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.],
          [0., 0.]],

         [[0., 0.],
          [0., 0.],
          [0., 0.]]]]])

<h2>Set Training Parameters</h2>

In [7]:
MIN_EXPLORE_RATE = 0.01
MIN_LEARNING_RATE = 0.1

NUM_EPISODES = 1000
MAX_T = 250
STREAK_TO_END = 120
SOLVED_T = 199
DEBUG_MODE = True

In [8]:
def get_explore_rate(t):
    if t >= 24:
        return max(MIN_EXPLORE_RATE, min(1, 1.0-math.log10((t+1)/25)))
    else:
        return 1.0

In [9]:
def get_learning_rate(t):
    if t >= 24:
        return max(MIN_LEARNING_RATE, min(0.5, 1.0 - math.log10((t+1)/25)))
    else:
        return 1.0

In [10]:
def select_action(state, explore_rate):
    # Select a random action
    if random.random() < explore_rate:
        action = env.action_space.sample()
    # Select the action with the highest q
    else:
        action = np.argmax(q_table[state])
    return action

In [11]:
def state_to_bucket(state):
    bucket_indice = []
    for i in range(len(state)):
        if state[i] <= STATE_BOUNDS[i][0]:
            bucket_index = 0
        elif state[i] >= STATE_BOUNDS[i][1]:
            bucket_index = NUM_BUCKETS[i]-1
        else:
            bound_width = STATE_BOUNDS[i][1] - STATE_BOUNDS[i][0]
            offset = (NUM_BUCKETS[i]-1)*STATE_BOUNDS[i][0]/bound_width
            scaling = (NUM_BUCKETS[i]-1)/bound_width
            bucket_index = int(round(scaling*state[i]-offset))
        bucket_indice.append(bucket_index)
    return tuple(bucket_indice)

In [12]:
explore_rate = get_explore_rate(0)
obv = env.reset()
state_0 = state_to_bucket(obv)
action = select_action(state_0, explore_rate)
print(state_0)
print(action)
print(q_table[state_0 + (action,)])

(0, 0, 3, 1)
0
0.0


<h2>Q Learning</h2>

In [13]:
learning_rate = get_learning_rate(0)
explore_rate = get_explore_rate(0)
discount_factor = 0.99

num_streaks = 0

for episode in range(NUM_EPISODES):
    # Reset env.
    obv = env.reset()
    print(obv)
    
    # initial state
    state_0 = state_to_bucket(obv)
    
    for t in range(MAX_T):
        env.render()
        
        # Select an action
        action = select_action(state_0, explore_rate)
        print(action)
        # Execute the action
        obv, reward, done, _ = env.step(action)
        
        # Observe the result
        state = state_to_bucket(obv)
        
        # Update Q based on the result
        best_q = np.amax(q_table[state])
        q_table[state_0 + (action,)] += learning_rate * (reward + discount_factor*(best_q) - q_table[state_0 + (action,)])
        
        # Setting up for the next iteration
        state_0 = state
        
#         # Print data$                                                       
#         if (DEBUG_MODE):                                                   
#             print("\nEpisode = %d" % episode)                              
#             print("t = %d" % t)                                            
#             print("Action: %d" % action)                                   
#             print("State: %s" % str(state))                                
#             print("Reward: %f" % reward)                                   
#             print("Best Q: %f" % best_q)                                   
#             print("Explore rate: %f" % explore_rate)                       
#             print("Learning rate: %f" % learning_rate)                     
#             print("Streaks: %d" % num_streaks)                             

#             print("")                                                      

        if done:                                                           
            print("Episode %d finished after %f time steps" % (episode, t)) 
            if (t >= SOLVED_T):                                            
                num_streaks += 1                                           
            else:                                                           
                num_streaks = 0                                            
            break

        # Update parameters
        explore_rate = get_explore_rate(episode)
        learning_rate = get_learning_rate(episode)
        

[ 0.03631789  0.00414612 -0.02002881 -0.00773128]
1
1
0
1
0
1
0
1
1
1
1
0
1
0
Episode 0 finished after 13.000000 time steps
[-0.04716803 -0.03757167  0.01706244  0.01471895]
0
1
0
1
0
1
0
1
0
1
1
0
1
1
0
1
1
1
0
0
1
0
0
1
0
0
0
1
0
0
1
0
1
1
0
1
0
0
0
1
0
0
0
1
Episode 1 finished after 43.000000 time steps
[ 0.01153851 -0.01163225  0.049721    0.04808353]
0
1
0
0
0
0
0
1
1
1
Episode 2 finished after 9.000000 time steps
[ 0.0185542   0.01504593  0.01884467 -0.01110786]
0
0
1
1
1
1
0
1
0
1
0
0
0
1
1
0
0
0
1
1
1
0
1
1
0
1
1
1
0
1
1
1
1
1
1
Episode 3 finished after 34.000000 time steps
[-0.03649035  0.02214883  0.00253543 -0.01897581]
0
0
0
0
1
1
0
1
1
1
0
0
1
0
0
1
Episode 4 finished after 15.000000 time steps
[-0.00141646  0.03894878  0.04340435 -0.01422048]
1
1
1
1
0
0
1
0
0
1
1
1
1
0
0
1
Episode 5 finished after 15.000000 time steps
[ 0.00715298 -0.01781306  0.00943    -0.01620888]
1
0
0
0
1
1
1
1
0
0
0
1
1
1
1
0
0
1
0
1
0
0
0
1
0
0
1
1
1
0
0
1
1
0
0
1
0
0
0
0
1
0
1
0
1
0
0
0
0
1
1
0
0

KeyboardInterrupt: 